# Telco Customer Churn — End-to-End Case Study

**Author:** Najam us Sahar
**Goal:** Build an end-to-end churn analysis from raw data - insights - modelling - recommendations.

Dataset: Telco Customer Churn (Excel).

## 1) Problem framing
**What is churn here?** A customer who **cancels service** (target column `Churn` = `Yes`).

**Why it matters:** churn reduces recurring revenue, increases acquisition costs (replacing lost customers), and often signals product/service gaps. Measuring churn helps prioritize retention interventions and forecast revenue risk.

In [ ]:
import pandas as pd
import numpy as np

DATA_PATH = "WA_Fn-UseC_-Telco-Customer-Churn 2.xlsx"
df = pd.read_excel(DATA_PATH)
df.shape, df.head()


## 2) Data cleaning & preparation
- Handle missing values (notably `TotalCharges` contains blanks).
- Fix data types.
- Validate the target (`Churn`).

In [ ]:
df_clean = df.copy()

# Fix TotalCharges: convert to numeric (blanks -> NaN)
df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")

missing = df_clean.isna().sum().sort_values(ascending=False)
missing.head(10)


In [ ]:
df_clean["Churn"].value_counts(), df_clean["Churn"].unique()


## 3) Exploratory Data Analysis (EDA)
We’ll compare churn vs non-churn customers across:
- tenure
- monthly charges
- contract type
- internet service type
- key add-on services (security/support)

### 3.1 Overall churn rate

In [ ]:
churn_rate = (df_clean["Churn"]=="Yes").mean()
churn_rate


In [ ]:
import matplotlib.pyplot as plt

# Tenure distribution by churn
plt.figure()
df_clean[df_clean["Churn"]=="No"]["tenure"].hist(bins=30, alpha=0.7, label="No")
df_clean[df_clean["Churn"]=="Yes"]["tenure"].hist(bins=30, alpha=0.7, label="Yes")
plt.title("Tenure distribution by churn")
plt.xlabel("Tenure (months)")
plt.ylabel("Customers")
plt.legend()
plt.show()


In [ ]:
# Monthly charges by churn
plt.figure()
df_clean[df_clean["Churn"]=="No"]["MonthlyCharges"].hist(bins=30, alpha=0.7, label="No")
df_clean[df_clean["Churn"]=="Yes"]["MonthlyCharges"].hist(bins=30, alpha=0.7, label="Yes")
plt.title("MonthlyCharges distribution by churn")
plt.xlabel("MonthlyCharges")
plt.ylabel("Customers")
plt.legend()
plt.show()


In [ ]:
# Churn rate by Contract / InternetService / PaymentMethod
def churn_by(col):
    return df_clean.groupby(col)["Churn"].apply(lambda s: (s=="Yes").mean()).sort_values(ascending=False)

for col in ["Contract","InternetService","PaymentMethod","TechSupport","OnlineSecurity"]:
    print("\n", col)
    display(churn_by(col))


## 4) Feature engineering
Create features that may help explain churn:
- `tenure_group` buckets
- `avg_monthly_spend` (TotalCharges/tenure with safe fallback)
- `services_count` (how many add-on services are active)


In [ ]:
fe = df_clean.copy()

fe["tenure_group"] = pd.cut(
    fe["tenure"],
    bins=[-1, 0, 12, 24, 48, 72],
    labels=["0","1-12","13-24","25-48","49-72"]
)

fe["avg_monthly_spend"] = fe["TotalCharges"] / (fe["tenure"].replace(0, np.nan))
fe["avg_monthly_spend"] = fe["avg_monthly_spend"].fillna(fe["MonthlyCharges"])

service_cols = [
    "PhoneService","MultipleLines","OnlineSecurity","OnlineBackup","DeviceProtection",
    "TechSupport","StreamingTV","StreamingMovies"
]

def yes_no_to_binary(s):
    return s.replace({"Yes":1,"No":0,"No phone service":0,"No internet service":0})

for c in service_cols:
    fe[c+"_bin"] = yes_no_to_binary(fe[c])

fe["services_count"] = fe[[c+"_bin" for c in service_cols]].sum(axis=1)

fe[["tenure","tenure_group","avg_monthly_spend","services_count"]].head()


## 5) Churn modelling
We will train **two** classification models:
- Logistic Regression (interpretable baseline)
- Random Forest (nonlinear model)

Evaluation:
- Confusion matrix
- Precision / Recall
- ROC-AUC

**Note:** The dataset is imbalanced (~26.5% churn), so we use `class_weight='balanced'` variants.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, RocCurveDisplay, PrecisionRecallDisplay,
    precision_score, recall_score, f1_score, accuracy_score
)

y = fe["Churn"].map({"Yes":1,"No":0})
X = fe.drop(columns=["Churn"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X_train.columns if c not in numeric_features]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

lr = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

rf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=400, random_state=42, class_weight="balanced_subsample",
        min_samples_leaf=5, n_jobs=-1
    ))
])

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

lr_proba = lr.predict_proba(X_test)[:,1]
rf_proba = rf.predict_proba(X_test)[:,1]
lr_pred = (lr_proba>=0.5).astype(int)
rf_pred = (rf_proba>=0.5).astype(int)

def summarize(y_true, y_pred, y_proba):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_proba),
        "confusion_matrix": confusion_matrix(y_true, y_pred),
    }

lr_sum = summarize(y_test, lr_pred, lr_proba)
rf_sum = summarize(y_test, rf_pred, rf_proba)

lr_sum, rf_sum


In [ ]:
print("Logistic Regression\n", classification_report(y_test, lr_pred))
print("Confusion matrix\n", lr_sum["confusion_matrix"])


In [ ]:
print("Random Forest\n", classification_report(y_test, rf_pred))
print("Confusion matrix\n", rf_sum["confusion_matrix"])


In [ ]:
import matplotlib.pyplot as plt
plt.figure()
RocCurveDisplay.from_predictions(y_test, lr_proba, name="LogReg")
RocCurveDisplay.from_predictions(y_test, rf_proba, name="RF")
plt.title("ROC Curves")
plt.show()

plt.figure()
PrecisionRecallDisplay.from_predictions(y_test, lr_proba, name="LogReg")
PrecisionRecallDisplay.from_predictions(y_test, rf_proba, name="RF")
plt.title("Precision-Recall Curves")
plt.show()


## 6) Postdictive analysis (predicted vs actual)
Where does the model do well and where does it fail?
- **False Negatives (FN):** customers who churned but the model missed them.
- **False Positives (FP):** customers predicted to churn but they did not.

We inspect probability scores and segment patterns for errors.

In [ ]:
pred_df = X_test.copy()
pred_df["actual_churn"] = y_test.values
pred_df["proba_churn"] = lr_proba
pred_df["pred_churn"] = lr_pred

pred_df["error_type"] = np.where(
    (pred_df["actual_churn"]==1) & (pred_df["pred_churn"]==0), "FN (missed churn)",
    np.where((pred_df["actual_churn"]==0) & (pred_df["pred_churn"]==1), "FP (false alarm)", "Correct")
)

pred_df["error_type"].value_counts()


In [ ]:
# Probability summary by error type
pred_df.groupby("error_type")["proba_churn"].agg(["count","mean","median"]).sort_values("mean", ascending=False)


In [ ]:
# Where do false negatives cluster?
for col in ["Contract","InternetService","PaymentMethod","tenure_group"]:
    print("\nFN share by", col)
    display(pred_df[pred_df["error_type"]=="FN (missed churn)"][col].value_counts(normalize=True))


## 7) Business recommendations
Based on EDA + model behaviour, typical high-risk segments include:
- Month-to-month contracts (highest churn)
- Fiber optic internet customers with low perceived value (high churn)
- Electronic check payers (highest churn)
- Low-tenure customers (first 12 months)
- Customers lacking **OnlineSecurity** and **TechSupport** add-ons

### Recommended actions
1. **Convert month-to-month → annual**: targeted offers at month 2–3 and month 10–12.
2. **New-customer onboarding** (first 90 days): proactive support calls / setup help.
3. **Value-pack bundles**: include security/support for fiber customers; test pricing.
4. **Payment friction reduction**: incentivize auto-pay; investigate electronic check pain points.
5. **Prioritize retention list**: score customers weekly; focus on top-decile risk.

### Operating the model
- Use churn probability thresholds depending on cost trade-offs.
- For retention campaigns, consider **lowering threshold** to catch more churners (reducing FN) if outreach is cheap.
- Track outcomes (saved vs churned) to continuously improve.